In [16]:
import sys
sys.path.append(sys.path[0][:sys.path[0].find('DVML-P7') + len('DVML-P7')])

import pandas as pd
from Code.UtilityFunctions.run_query import run_query
from Code.UtilityFunctions.get_data_path import get_path
from Code.UtilityFunctions.wikidata_functions import get_name_of_location_with_long_lat, wikidata_query

In [2]:
biz = pd.read_json(get_path("yelp_academic_dataset_business.json"), lines=True)

coordinates = (biz["longitude"].apply(round, args=(2,)).astype(str) + "," + biz["latitude"].apply(round, args=(2,)).astype(str)).unique()
coordinates

array(['-119.71,34.43', '-90.34,38.55', '-110.88,32.22', ...,
       '-90.05,30.02', '-90.23,38.66', '-75.41,40.2'], dtype=object)

In [3]:
coordinates[0:10]

array(['-119.71,34.43', '-90.34,38.55', '-110.88,32.22', '-75.16,39.96',
       '-75.47,40.34', '-87.06,36.27', '-90.34,38.63', '-82.73,27.77',
       '-90.32,38.57', '-86.77,36.21'], dtype=object)

In [4]:
coordinates[0:1][0]

'-119.71,34.43'

In [47]:
coordinates_df = pd.DataFrame(coordinates, columns=['coordinates']).head(1)

In [6]:
get_name_of_location_with_long_lat(coordinates[0])

('Q159288', 'Santa Barbara')

In [7]:
for i in coordinates_df.itertuples():
    if get_name_of_location_with_long_lat(i.coordinates) is None:
        print(i.coordinates, get_name_of_location_with_long_lat(i.coordinates))

In [8]:
df = pd.DataFrame(coordinates_df['coordinates'].apply(lambda x: [x] + list(get_name_of_location_with_long_lat(x))).tolist(), columns=['coordinates','mapped_qid', 'qid_label'])
df.to_csv(path_or_buf='/home/ubuntu/DVML-P7/Development/location_mappings.csv',index=False)

In [9]:
df

,coordinates,mapped_qid,qid_label
0,"-119.71,34.43",Q159288,Santa Barbara


In [10]:
location_mappings = pd.read_csv(get_path("location_mappings.csv"))

In [11]:
location_mappings

,coordinates,mapped_qid,qid_label
0,"-119.71,34.43",Q159288,Santa Barbara
1,"-90.34,38.55",Q966126,Grantwood Village
2,"-110.88,32.22",Q18575,Tucson
3,"-75.16,39.96",Q1345,Philadelphia
4,"-75.47,40.34",Q1183476,Green Lane
...,...,...,...
11410,"-82.68,27.87",Q2151669,Pinellas Park
11411,"-119.8,39.32",Q7972623,Washoe Valley
11412,"-90.05,30.02",Q34404,New Orleans
11413,"-90.23,38.66",Q38022,St. Louis


In [17]:
print(f"Number of unique qid's: {len(location_mappings.mapped_qid.unique())}\nNumber of unique city names: {len(location_mappings.qid_label.unique())}")

Number of unique qid's: 693
Number of unique city names: 684


In [41]:
import numpy as np
location_mappings[location_mappings.mapped_qid.isna()]

,coordinates,mapped_qid,qid_label
745,"-119.51,34.6",NaN,NaN


In [51]:
location_mappings_df = pd.DataFrame(coordinates_df['coordinates'].apply(lambda x: [x] + list(get_name_of_location_with_long_lat(x))).tolist(), columns=['coordinates','city', 'cityLabel', 'cityPopulation', 'county', 'countyLabel', 'state', 'stateLabel', 'country', 'countryLabel'])

In [46]:
get_name_of_location_with_long_lat(coordinates[0])

('Q159288',
 'Santa Barbara',
 '88665',
 'Q108106',
 'Santa Barbara County',
 'Q99',
 'California',
 'Q30',
 'United States of America')

In [52]:
location_mappings_df

,coordinates,city,cityLabel,cityPopulation,county,countyLabel,state,stateLabel,country,countryLabel
0,"-119.71,34.43",Q159288,Santa Barbara,88665,Q108106,Santa Barbara County,Q99,California,Q30,United States of America
